In [9]:
from bs4 import BeautifulSoup as bs
import requests
import json
import numpy as np
import pandas as pd
#import pymongo
import base64
import bson
from bson.binary import Binary
import os
import pdfplumber
from urllib.parse import unquote
import re

## Web scrapping

In [10]:
Domain='http://www.bvmt.com.tn/'
URL='http://www.bvmt.com.tn/fr/ab-etats-financiers-semestriels-30-06-2020'
FILETYPE='.pdf'
URL0="http://www.bvmt.com.tn/fr/entreprises/list"


In [11]:
def get_soup(url):
    return bs(requests.get(url).text,'html.parser')

In [12]:
ListID = []
for link in get_soup(URL0).find_all('option'):
    file_link = link.get('uri')
    #print(file_link)
    if file_link is not None:
        new=file_link.replace('/fr/profile-emetteur_profile/','')
        print(new)
        ListID.append(new)
        file_name = link.text

93
68
134
86
87
136
64
70
58
24
49
41
50
1528
583
42
57
26
43
44
46
60
83
1174
9258
51
82
1124
8574
77
39
78
184
71
183
88
135
38
1566
65
1171
185
133
10708
54
76
1170
12625
12356
63
74
84
79
69
80
81
37
1173
62
1274
90
66
85
55
59
45
72
73
1278
91
89
56
61
67
53
10028
47
48
11547
52


In [ ]:
print(ListID)

In [14]:
# Recupere les liens de chaque entreprise
urls = []
urls1 = []
urls3 = []

for item in ListID:
    site= "http://www.bvmt.com.tn/fr/documents/"+item+"/18/list"
    s = get_soup(site)
    
    #verifie si l'entreprise a un bilan consolidé
    pdf0=s.find("div", {'onclick' : re.compile("consolid",re.IGNORECASE)})

    #print(pdf0)
    
    #si l'entreprise n'a pas de bilan consolidé
    if(pdf0 is None):
        print("pas de bilan consolidé")
    #sinon ajoute le lien de l'entreprise dans la liste
    else :
        print("bilan ajouté")
        pdf1=pdf0.get('onclick')
        newpdf=pdf1.replace('document.location.href=\'','')
        newpdf=newpdf.replace('\'','')
    
        urls3.append(newpdf)
    
 
                   
        #if href.startswith("/fr/content/cours"):
            #urls.append(href)

bilan ajouté
pas de bilan consolidé
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
pas de bilan consolidé
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
bilan ajouté
bilan ajouté
pas de bilan consolidé
bilan ajouté
bilan ajouté
pas de bilan consolidé
pas de bilan consolidé
pas de bilan consolidé
bilan ajouté
pas de bilan consolidé
pas de bilan consolidé
bila

In [ ]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        assert r.status_code == 200, f'error, status code is {r.status_code}'
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [15]:
#Cherhce tout les lien pdf

#dictionnaire pdfList
pdfList = {}
for link in urls3:
    pdfPage = get_soup('http://www.bvmt.com.tn'+link)
    
    #cherche le lien de telechargement pdf
    downLoad = pdfPage.find("a", {'href' : re.compile("/default/files/societes/",re.IGNORECASE)})
    
    pdfDownload = downLoad.get('href')
    #trouve le nom de la societé
    compnyName = pdfPage.find('h3').text.strip().split('-',1)
    print(compnyName[0])
    #dictionnaire clé:nomSocieté  valeur:lien pdf
    pdfList[compnyName[0]] = pdfDownload
    

Amen Bank 
AIR LIQUIDE TUNISIE 
SOCIETE CHIMIQUE ALKIMIA 
Société Les Ateliers Mécaniques du Sahel 
AUTOMOBILE RESEAU TUNISIEN ET SERVICES 
SOCIETE L'ACCUMULATEUR TUNISIEN 
ARAB TUNISIAN BANK 
ARAB TUNISIAN LEASE 
BANQUE ATTIJARI DE TUNISIE 
ASSURANCES MAGHREBIA 
BH BANK 
BIAT 
BANQUE NATIONALE AGRICOLE 
BANQUE DE TUNISIE 
BANQUE DE TUNISIE ET DES EMIRATS  
Société Cellcom 
CEREALIS 
COMPAGNIE INTERNATIONALE DE LEASING 
CITY CARS 
DELICE HOLDING 
ELECTROSTAR 
ENNAKL AUTOMOBILES 
ESSOUKNA 
EURO
SOCIETE GENERALE INDUSTRIELLE DE FILTRATION 
HANNIBAL LEASE SA 
Société « LAND'OR » 
MAGASIN GENERAL 
Maghreb International Publicité 
SOCIETE NOUVELLE MAISON DE LA VILLE DE TUNIS 
MPBS « Manufacture de Panneaux Bois du Sud » 
STE ONE TECH HOLDING SA 
OFFICE PALST 
POULINA GROUP HOLDING 
Société d’Articles Hygiéniques SAH SA  
SAM 
SERVICOM 
Société de Fabrication des Boissons de Tunisie 
STE IND. D'APPAREILLAGE ET DE MATERIELS ELEC. 
SOCIETE IMMOBILIERE ET DE PARTICIPATIONS 
SOCIETE IMMOBILIERE 

In [23]:
print (len(pdfList))

55


In [ ]:
import urllib.request

In [ ]:
#telecharge tous les pdfs
i = 0
for link in pdfList: 
    
    try:
        conn = urllib.request.urlopen(link)
    except urllib.error.HTTPError as e:
        # Return code error (e.g. 404, 501, ...)
        # ...
        print('HTTPError: {}'.format(e.code))
        
    except urllib.error.URLError as e:
        # Not an HTTP-specific error (e.g. connection refused)
        # ...
        print('URLError: {}'.format(e.reason))
        
    else:
        # 200
        # ...
        urllib.request.urlretrieve(link, "etatFinancier"+str(i)+".pdf")
        i+=1
        print('good')
    

    #invoice_pdf = download_file(invoice)

### stockage dans la base

In [6]:
import pymongo
from pymongo import MongoClient

In [7]:
#Connection avec la base de données
cluster = MongoClient('mongodb://localhost:27017/')
db = cluster['Finance']
collection = db['DocuPdf']


In [20]:
#Ajoute dans la base de données
count = 0
postList = []
for key, value in pdfList.items():
    count += 1;
    postList.append({'_id':count,'companie' : key,'lien':value })

collection.insert_many(postList)
    

## Preparation de données

In [4]:
import cv2
from PIL import Image
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

<function pytesseract.pytesseract.image_to_string(image, lang=None, config='', nice=0, output_type='string', timeout=0)>

In [6]:
# Path of the pdf 
PDF_file = "etat.pdf"

    
''' 
Part #1 : Converting PDF to images 
'''
#convert_from_path(poppler_path=r'C:\Users\anaconda3\envs\poppler\bin')
  
# Store all the pages of the PDF in a variable 
pages = convert_from_path(PDF_file, 500,poppler_path = r'C:\Users\Nebil\poppler\poppler-0.68.0\bin') 
  
# Counter to store images of each page of PDF to image 
image_counter = 1
  
# Iterate through all the pages stored above 
for page in pages: 
  
    # Declaring filename for each page of PDF as JPG 
    # For each page, filename will be: 
    # PDF page 1 -> page_1.jpg 
    # PDF page 2 -> page_2.jpg 
    # PDF page 3 -> page_3.jpg 
    # .... 
    # PDF page n -> page_n.jpg 
    filename = "page_"+str(image_counter)+".jpg"
      
    # Save the image of the page in system 
    page.save(filename, 'JPEG') 
  
    # Increment the counter to update filename 
    image_counter = image_counter + 1
 

In [7]:
 
''' 
Part #2 - Recognizing text from the images using OCR 
'''
    
# Variable to get count of total number of pages 
filelimit = image_counter-1
  
# Creating a text file to write the output 
outfile = "out_text.txt"
  
# Open the file in append mode so that  
# All contents of all images are added to the same file 
f = open(outfile, "a") 
  
# Iterate from 1 to total number of pages 
for i in range(1, filelimit + 1): 
  
    # Set filename to recognize text from 
    # Again, these files will be: 
    # page_1.jpg 
    # page_2.jpg 
    # .... 
    # page_n.jpg 
    filename = "page_"+str(i)+".jpg"
          
    # Recognize the text as string in image using pytesserct 
    text = str(((pytesseract.image_to_string(Image.open(filename))))) 
  
    # The recognized text is stored in variable text 
    # Any string processing may be applied on text 
    # Here, basic formatting has been done: 
    # In many PDFs, at line ending, if a word can't 
    # be written fully, a 'hyphen' is added. 
    # The rest of the word is written in the next line 
    # Eg: This is a sample text this word here GeeksF- 
    # orGeeks is half on first line, remaining on next. 
    # To remove this, we replace every '-\n' to ''. 
    text = text.replace('-\n', '')     
  
    # Finally, write the processed text to the file. 
    f.write(text) 
  
# Close the file after writing all the text. 
f.close() 

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [ ]:
import csv
with open('delice.csv', 'w', newline='') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(lines)

In [ ]:
# establish a connection to the database
connection = pymongo.MongoClient()

#get a handle to the test database
db = connection.test
file_meta = db.file_meta
file_used = file_name

def main():
    coll = db.sample
    with open(file_used, "rb") as f:
        encoded = Binary(f.read())

    coll.insert({"filename": file_used, "file": encoded, "description": "test" })


In [ ]:
from pymongo import MongoClient
import gridfs

db = MongoClient('mongodb://localhost:27017/').myDB
fs = gridfs.GridFS( db )
fileID = fs.put( open((file_name)  ))
out = fs.get(fileID)

In [ ]:
from pymongo import MongoClient 
  
try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

In [ ]:
# database 
db = conn.database 
  
# Created or Switched to collection names: my_gfg_collection 
collection = db.my_gfg_collection 
  
emp_rec1 = x

  
# Insert Data 
rec_id1 = collection.insert_one(emp_rec1) 

  
print("Data inserted with record ids",rec_id1) 
  
# Printing the data inserted 
cursor = collection.find() 
for record in cursor: 
    print(record) 